Importing the Dataset

In [ ]:
import numpy as np
import os, glob, pandas as pd

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/heart-failure-clinical-data")

print("Path to dataset files:", path)

In [ ]:
df = pd.read_csv(os.path.join(path, "heart_failure_clinical_records_dataset.csv"))
df.head()

Checking for any Null Values (Data Cleaning)

In [ ]:
df.isna().sum()

In [ ]:
df.shape

Checking for Outliers (Data Cleaning)

In [ ]:
non_binary_columns = ['creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium']
quartile_1 = df[non_binary_columns].quantile(0.25)
quartile_3 = df[non_binary_columns].quantile(0.75)
iqr = quartile_3 - quartile_1
lower_bound = quartile_1 - 1.5 * iqr
upper_bound = quartile_3 + 1.5 * iqr

outliers = (df[non_binary_columns] < lower_bound) | (df[non_binary_columns] > upper_bound)
outliers_count = outliers.sum()

print("Number of outliers in each column:")
print(outliers_count)


In [ ]:
# helper function for evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def evaluate_model(y_true, y_pred, y_proba=None, model_name="Model"):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred))

    if y_proba is not None:
        print("ROC-AUC:", roc_auc_score(y_true, y_proba))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

# EDA Notebook 01
We will be using this notebook for preliminary EDA to understand the data better before we start fitting models and analyzing results


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style of the plots
sns.set_style('darkgrid')


In [ ]:
print(df.head(1))
print('='*100)
print(df.info())
print('='*100)
print(df.describe())
print('='*100)
# Check for missing values
print(df.isnull().sum())

### Clean Data (some Preprocessing)
Note: we should still use non scaled (raw) data for initial EDA


In [ ]:
continuous_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

In [ ]:
# Remove Outliers
quantile_1 = df[continuous_features].quantile(0.25)
quantile_3 = df[continuous_features].quantile(0.75)
iqr = quantile_3 - quantile_1
lower_bound = quantile_1 - 1.5 * iqr
upper_bound = quantile_3 + 1.5 * iqr

outliers = (df[continuous_features] < lower_bound) | (df[continuous_features] > upper_bound)

print("Outliers: ")
outliers.sum()

In [ ]:
rows_w_outliers = outliers.any(axis=1)
df_clean = df[~rows_w_outliers] # return dataframe without outliers

df_clean.shape
df = df_clean.copy()

In [ ]:
# save cleaned data to csv
# df.to_csv('../data/heart_failure_clinical_records_dataset_cleaned.csv', index=False)

There are not any missing values, we should be okay to proceed to EDA

### Explore DEATH_EVENT

In [ ]:
# Explore death event distribution
plt.figure(figsize=(10, 6))
sns.countplot(x='DEATH_EVENT', data=df)
plt.title('Distribution of Death Events')
plt.xlabel('Death Event')
plt.ylabel('Count')
plt.xticks([0, 1], ['Survived', 'Died'])
plt.show()
print("Percentage of DEATH_EVENT:")
print(df['DEATH_EVENT'].value_counts(normalize=True)*100)
imbalance_ratio = df['DEATH_EVENT'].value_counts(normalize=True)[0] / df['DEATH_EVENT'].value_counts(normalize=True)[1]
print(f"\nImbalance ratio: {imbalance_ratio:.2f}")



The dataset shows moderate class imbalance, with approximately 68% survival rate (203 patients) and 32% mortality rate (96 patients). The imbalance ratio is about 2:1, which is moderate - not severe enough to require aggressive resampling, but we should consider using stratified cross-validation and monitoring precision/recall metrics in addition to accuracy.

In [ ]:
# Death Rate between Male and Female:

male = df.loc[df['sex'] == 1, 'DEATH_EVENT']
female = df.loc[df['sex'] == 0, 'DEATH_EVENT']

print("Male Death Rate", male.mean())
print("Female Death Rate", female.mean())

### Univariate EDA

In [ ]:
# Create histograms and KDE for continuous features
fix, axis = plt.subplots(3,3, figsize=(15,12))
axis = axis.flatten()

for i, feature in enumerate(continuous_features):
    sns.histplot(
        df[feature],
        ax=axis[i],
        kde=True,
        bins=30)
    axis[i].set_title(f'Histogram and KDE of {feature}')
    axis[i].set_xlabel(feature)
    axis[i].set_ylabel('Frequency')

    skew = df[feature].skew()
    axis[i].text(0.7, 0.9, f'Skewness: {skew:.2f}',
        transform=axis[i].transAxes,
        bbox=dict(boxstyle='round',facecolor='wheat',alpha=0.5))

axis[-2].remove()
axis[-1].remove()
plt.tight_layout()
plt.show()

### Analysis of Feature Distributions (After Outlier Removal)

### Skewness Interpretation

**Highly Symmetric (|skew| < 0.5):**
- **time** (skew = 0.06): Nearly perfectly symmetric distribution
- **serum_sodium** (skew = -0.11): Almost perfectly symmetric, well-centered distribution
- **platelets** (skew = 0.25): Very symmetric, minimal skew
- **age** (skew = 0.37): Slight right skew, relatively normal distribution centered around 60 years
- **ejection_fraction** (skew = 0.38): Slight right skew, appears somewhat uniform with potential subgroups (normal vs reduced ejection fraction)

**Moderately Skewed (0.5 < |skew| < 2):**
- **serum_creatinine** (skew = 0.94): Moderate right skew, most patients have normal kidney function with some elevated values
- **creatinine_phosphokinase** (skew = 0.97): Moderate right skew, previously highly skewed but much improved after outlier removal

### Key Observations

**Impact of Outlier Removal:**
1. **Dramatic improvement in skewness**: CPK and serum_creatinine went from extremely skewed (4.46) to moderately skewed (~0.95), making them much more suitable for parametric models
2. **Better normality**: Most features now fall within the fairly symmetric range (|skew| < 0.5), which is ideal for many machine learning algorithms
3. **Preserved clinical validity**: Distributions still reflect real medical patterns without extreme outliers

**Remaining Characteristics:**
- **Scale differences**: Features still span different ranges (will require normalization/standardization for modeling)
- **Potential subgroups**: Ejection fraction still shows evidence of two patient populations (normal vs reduced EF)
- **Data quality**: After outlier removal, the dataset is cleaner and more suitable for modeling while maintaining clinical relevance


### Bivariate EDA: Features vs Target (DEATH_EVENT)


In [ ]:
# Violin plots for key continuous features (showing distribution shape)
key_features = ['ejection_fraction', 'serum_creatinine', 'time', 'age']
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, feature in enumerate(key_features):
    sns.violinplot(data=df, x='DEATH_EVENT', y=feature, ax=axes[i], palette='muted', inner='quartile')
    axes[i].set_title(f'Violin Plot: {feature} vs DEATH_EVENT', fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Death Event (0=Survived, 1=Died)')
    axes[i].set_ylabel(feature)
    axes[i].set_xticklabels(['Survived', 'Died'])

plt.tight_layout()
plt.show()


### Analysis
#### Ejection Fraction:
Survived group:  

- Centered around 40–50%, with many patients between 35–60%.  
- Distribution is wider (more patients with moderately healthy function).  

Dead group:  

- Strongly centered near 25–30%.  
- Very few patients with healthy ejection fractions (>40%).  
- Shape is narrow and skewed toward low EF.  

*Conclusion:*  
Lower ejection fraction is associated with death.  
This feature will likely be one of the top predictors.  

#### Serum Creatinine

Survived:

- Most patients have creatinine around 0.8–1.2 mg/dL (normal range).
- Very tight distribution.

Died:

- Distribution shifts upward: many near 1.5–2.0 mg/dL.
- Much wider spread; more high-creatinine outliers.

*Conclusion:*  
Higher serum creatinine (worse kidney function) is strongly associated with death.
Expecting this feature to rank highly in model importance.

#### Age
- Patients who died tend to be older, which implies that age is a meaninful risk factor


In [ ]:
# Grouped bar charts: Death rate by categorical features
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, feature in enumerate(categorical_features):
    death_rate = df.groupby(feature)['DEATH_EVENT'].agg(['sum', 'count', 'mean'])
    death_rate['survival_rate'] = 1 - death_rate['mean']

    x = death_rate.index.astype(str)
    width = 0.35
    x_pos = np.arange(len(x))

    axes[i].bar(x_pos - width/2, death_rate['survival_rate'] * 100, width,
                label='Survived %', color='skyblue', edgecolor='black')
    axes[i].bar(x_pos + width/2, death_rate['mean'] * 100, width,
                label='Died %', color='salmon', edgecolor='black')

    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Percentage')
    axes[i].set_title(f'Death Rate by {feature}')
    axes[i].set_xticks(x_pos)
    axes[i].set_xticklabels([f'{feature}={val}' for val in x])
    axes[i].legend()
    axes[i].set_ylim(0, 100)

    for j, (surv, died) in enumerate(zip(death_rate['survival_rate'] * 100, death_rate['mean'] * 100)):
        axes[i].text(j - width/2, surv + 2, f'{surv:.1f}%', ha='center', fontsize=9)
        axes[i].text(j + width/2, died + 2, f'{died:.1f}%', ha='center', fontsize=9)

axes[-1].remove()
plt.tight_layout()
plt.show()


In [ ]:
# Statistical summary: Death rates by categorical features
print("Death Rates by Categorical Features:\n" + "="*60)
for feature in categorical_features:
    print(f"\n{feature}:")
    cross_tab = pd.crosstab(df[feature], df['DEATH_EVENT'], normalize='index') * 100
    cross_tab.columns = ['Survived %', 'Died %']
    print(cross_tab.round(2))

    # Chi-square test for independence
    from scipy.stats import chi2_contingency
    contingency_table = pd.crosstab(df[feature], df['DEATH_EVENT'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    print(f"  Chi-square test: χ² = {chi2:.2f}, p-value = {p_value:.4f} {'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'}")
    print("-" * 60)


### Correlation Analysis & Feature Interactions


In [ ]:
# Compute Pearson correlation matrix for all numeric features
correlation_matrix = df.corr(method='pearson')

# Create heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix,
            annot=True,
            fmt='.2f',
            cmap='coolwarm',
            center=0,
            square=True,
            linewidths=1,
            cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix (Pearson) - All Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()


In [ ]:
# Correlation with target variable (DEATH_EVENT)
target_corr = correlation_matrix['DEATH_EVENT'].drop('DEATH_EVENT').sort_values(ascending=False)

print("Correlations with DEATH_EVENT (sorted by absolute value):\n")
print("="*60)
for feature, corr_value in target_corr.items():
    strength = 'Strong' if abs(corr_value) > 0.5 else 'Moderate' if abs(corr_value) > 0.3 else 'Weak'
    direction = 'positive' if corr_value > 0 else 'negative'
    print(f"{feature:30s}: {corr_value:6.3f} ({strength} {direction})")

# Visualize correlations with DEATH_EVENT
plt.figure(figsize=(10, 8))
colors = ['red' if x < 0 else 'green' for x in target_corr.values]
target_corr.plot(kind='barh', color=colors, edgecolor='black')
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title('Feature Correlations with DEATH_EVENT', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.axvline(x=0.3, color='gray', linestyle='--', linewidth=0.5, alpha=0.7, label='Moderate threshold (±0.3)')
plt.axvline(x=-0.3, color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Check for multicollinearity among features (excluding target)
feature_corr = correlation_matrix.drop('DEATH_EVENT', axis=0).drop('DEATH_EVENT', axis=1)

# Find pairs with high correlation (potential multicollinearity)
print("\nHigh Feature-to-Feature Correlations (|r| > 0.5):")
print("="*60)
high_corr_pairs = []
for i in range(len(feature_corr.columns)):
    for j in range(i+1, len(feature_corr.columns)):
        if abs(feature_corr.iloc[i, j]) > 0.5:
            high_corr_pairs.append((feature_corr.columns[i],
                                   feature_corr.columns[j],
                                   feature_corr.iloc[i, j]))
            print(f"{feature_corr.columns[i]:30s} <-> {feature_corr.columns[j]:30s}: {feature_corr.iloc[i, j]:6.3f}")

if not high_corr_pairs:
    print("No strong feature-to-feature correlations detected (good - low multicollinearity)")
else:
    print(f"\nFound {len(high_corr_pairs)} pairs with potential multicollinearity")


### Note on PCA:

We did not apply PCA because the dataset contains only twelve features, all of which are clinically meaningful and interpretable. PCA is most useful for high-dimensional datasets or when features are highly correlated, neither of which applies here. Additionally, PCA removes interpretability by transforming predictors into abstract components, and several of our models (such as logistic regression with regularization and tree-based methods) already handle correlation and noise effectively without dimensionality reduction.


# Data Cleaning & Preprocessing Notebook 02
This is where we will do any cleaning and standardizations to prepare for using ML models to predict DEATH_EVENT.  

*Note:* Some of these steps are redundant/unnecessary but are show for 'full lifecycle coverage'. Sometimes these two note books (ie. 01,02) are combined into 1 for simplicity.  

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# Set the style of the plots
sns.set_style('darkgrid')

In [ ]:
continuous_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

In [ ]:
# Detect and handle missing values (already checked)
missing_values = df.isnull().sum()
print("Missing values:")
print(missing_values)

# Handle missing values - None


In [ ]:
# Remove Outliers (already checked)
quantile_1 = df[continuous_features].quantile(0.25)
quantile_3 = df[continuous_features].quantile(0.75)
iqr = quantile_3 - quantile_1
lower_bound = quantile_1 - 1.5 * iqr
upper_bound = quantile_3 + 1.5 * iqr

outliers = (df[continuous_features] < lower_bound) | (df[continuous_features] > upper_bound)

print("Outliers: ")
outliers.sum()
# Handle Outliers
# Will skip this step as we did it in 01 EDA notebook
# (ignoring the 3 extra outliers since they are being calculate on the cleaned dataset)

### Standardization and Train Split
Some of our ML models that we will be using require standardization of features. So we will first split our data, then standardize it.  

Process:  
- We will standardize only the continuous numerical features (age, creatinine_phosphokinase, ejection_fraction, platelets, serum_creatinine, serum_sodium, time) using StandardScaler.  

- Binary 0/1 variables were left unchanged.

- Standardization was applied in a scikit-learn Pipeline, fit only on the training data, and then applied to the test set.   

- This was necessary for scale-sensitive models such as logistic regression, SVM, k-NN, and neural networks.

- Tree-based models do not require scaling, but using a consistent preprocessing pipeline prevents data leakage and maintains comparability across models.



### Models That Require Feature Scaling

| **Model** | **Why It Needs Scaling** |
|----------|---------------------------|
| **Logistic Regression** | L2 regularization assumes all features are on the same scale; otherwise coefficients are penalized inconsistently. |
| **SVM (Linear, RBF kernels)** | Uses distance and dot products; features with larger magnitude dominate the decision boundary. |
| **k-NN** | Distance-based algorithm—unscaled features distort nearest-neighbor calculations. |
| **Neural Networks (PyTorch)** | Gradient-based optimization converges faster when inputs have similar scale; prevents exploding/vanishing gradients. |
| **PCA** | Computes variance along components; features with larger variance dominate unless scaled. |


In [ ]:
# Train, Validation, Test Split
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Column Transformer + Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_features),
        ('cat', 'passthrough', categorical_features)
    ]
)

# Example pipeline with logistic regression, will redo this part in the ML notebook
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(penalty='l2', C=0.1, solver='liblinear',class_weight='balanced'))]
)

In [ ]:
# Verify Feature Scaling
X_train_scaled = preprocessor.fit_transform(X_train)
print("Mean: \n", X_train_scaled.mean(axis=0))
print("Std: \n", X_train_scaled.std(axis=0))


Since for continous features the mean ~ 0 and std ~ 1, scaling is working as intended!

### Note:
The imbalance in the dataset in DEATH_EVENT is about 2.67:1 to died:survived. The imbalance is mild and using a process such as SMOTE may add unneccessary noise and can hurt generalization.

In [ ]:
# Add data to CSVs for access in next notebook
X_train.to_csv("X_train.csv", index=False)
X_val.to_csv("X_val.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_val.to_csv("y_val.csv", index=False)

# ML & Generating Predictions Notebook 03
This notebook is where we will evaluate baseline and candidate models to see which model we should use as our primary.  

*Note: We did not perform feature selection because the dataset contains only twelve clinically meaningful features, and removing any of them could discard useful non-linear or interaction effects. Simple correlation is not a reliable indicator of predictive value, especially in medical data. Several models we use, including Random Forests, Gradient Boosting, and regularized Logistic Regression, already perform built-in feature selection or regularization by down-weighting weaker predictors. Because these models naturally handle feature relevance, explicit feature elimination is unnecessary for this project. We will analyze feature importance in a future step*

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# XGBoost
from xgboost import XGBClassifier

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix,
    ConfusionMatrixDisplay
)

# Set plot style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:
# Results Dict
all_results = {}

### We will create a baseline logistic regression for our actual models to outperform

In [ ]:
continuous_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

In [ ]:
# Load data - splits were created in previous notebook
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv").values.ravel()  # Convert to 1D array

X_val = pd.read_csv("X_val.csv")
y_val = pd.read_csv("y_val.csv").values.ravel()  # Convert to 1D array

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Class distribution in training: {np.bincount(y_train)}")
print(f"Class distribution in validation: {np.bincount(y_val)}")

In [ ]:
# create pipeline and run logistic regression

# Column Transformer + Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_features),
        ('cat', 'passthrough', categorical_features)
    ]
)

# Example pipeline with logistic regression, will redo this part in the ML notebook
baseline_lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear',class_weight='balanced'))]
)

In [ ]:
# Train the baseline model
baseline_lr_pipeline.fit(X_train, y_train)

# Make predictions on VALIDATION set
y_val_pred = baseline_lr_pipeline.predict(X_val)
y_val_pred_proba = baseline_lr_pipeline.predict_proba(X_val)[:, 1]  # Probability of death

print("Baseline Logistic Regression trained successfully!")


In [ ]:
# Evaluate the Baseline Logistic Regression
evaluate_model(y_val, y_val_pred, y_val_pred_proba, model_name="Baseline Logistic Regression")


In [ ]:
def plot_roc_curve(y_true, y_pred_proba, model_name="Model"):
    """
    Plot ROC curve for a single model.

    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred_proba : array-like
        Predicted probabilities for positive class
    model_name : str
        Name of the model for display
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    roc_auc = roc_auc_score(y_true, y_pred_proba)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'{model_name} (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier (AUC = 0.500)')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate (Recall)', fontsize=12)
    plt.title(f'ROC Curve: {model_name}', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    return roc_auc


def plot_multiple_roc_curves(models_dict, y_true):
    """
    Plot multiple ROC curves on the same plot for comparison.

    Parameters:
    -----------
    models_dict : dict
        Dictionary with format: {'Model Name': y_pred_proba, ...}
    y_true : array-like
        True labels
    """
    plt.figure(figsize=(10, 8))

    colors = ['darkorange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

    for i, (model_name, y_pred_proba) in enumerate(models_dict.items()):
        fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
        roc_auc = roc_auc_score(y_true, y_pred_proba)
        color = colors[i % len(colors)]
        plt.plot(fpr, tpr, lw=2, color=color, label=f'{model_name} (AUC = {roc_auc:.3f})')

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier (AUC = 0.500)')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate (Recall)', fontsize=12)
    plt.title('ROC Curves: Model Comparison', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
# Plot ROC curve for baseline logistic regression
plot_roc_curve(y_val, y_val_pred_proba, model_name="Baseline Logistic Regression")


### Now we will use more complex models

Notes:  
- We used 5-fold stratified cross-validation for model selection because the dataset is relatively small (299 samples). Five folds provide a good balance between bias and variance, produce stable performance estimates, and avoid the computational cost of higher-fold CV while still giving more reliable results than a single train/validation split.
- We will use GridSearchCV to return the best model for each and to tune hyperparameters

### Logistic Regression
We will begin with logistic regression. We will:
- Tune the regularization strength of logistic regression using GridSearchCV over C ∈ {0.01, 0.1, 1, 10, 100} with 5-fold cross-validation and ROC–AUC as the scoring metric.
- Selected the C_i that produced the highest mean cross-validated ROC-AUC on thr training set.
- Refit the model with optimal C

In [ ]:
# Set up the pipeline
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', class_weight='balanced', max_iter=1000, random_state=42,penalty='l2'))
])

# Define the hyperparameter grid - trying different regularization strengths
c_vals = [i * 0.0001 for i in range(1, 15)]
c_vals.append(0.01)
c_vals.append(0.1)
param_grid = {
    'classifier__C': c_vals  # Lower C = stronger regularization
}

print(f"Testing {len(c_vals)} different C values with 5-fold CV")

# Setup 5-fold cross validation (stratified to keep class balance in each fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create GridSearchCV - this will try all C values and pick the best one
grid_search = GridSearchCV(
    estimator=lr_pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='roc_auc',  # Most important metric for us
    n_jobs=-1,  # Use all CPU cores
    verbose=0
)

# Fit - this runs all the cross-validation
grid_search.fit(X_train, y_train)

print("\nGrid search complete!")


In [ ]:
print("Best parameters found:")
print(grid_search.best_params_)
print(f"\nBest cross-validated ROC-AUC score: {grid_search.best_score_:.4f}")

# Look at all the results to see how each C performed
results_df = pd.DataFrame(grid_search.cv_results_)
print("\nAll cross-validation results:")
print(results_df[['param_classifier__C', 'mean_test_score', 'std_test_score']].sort_values('mean_test_score', ascending=False))


In [ ]:
# Visualize how performance changes with C
plt.figure(figsize=(10, 6))
mean_scores = results_df['mean_test_score'].values
std_scores = results_df['std_test_score'].values

plt.semilogx(c_vals, mean_scores, 'o-', linewidth=2, markersize=8, label='Mean ROC-AUC')
plt.fill_between(c_vals, mean_scores - std_scores, mean_scores + std_scores, alpha=0.2, label='± 1 std dev')
plt.xlabel('Regularization Parameter (C)', fontsize=12)
plt.ylabel('Cross-Validated ROC-AUC', fontsize=12)
plt.title('Logistic Regression: Hyperparameter Tuning Results', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Interesting to see - lower C means more regularization (simpler model)
# Higher C means less regularization (more complex model)


In [ ]:
# The best model is already refitted on the full training set by GridSearchCV
# We can access it directly
tuned_lr_model = grid_search.best_estimator_

# Make predictions on validation set
y_val_pred_tuned = tuned_lr_model.predict(X_val)
y_val_pred_proba_tuned = tuned_lr_model.predict_proba(X_val)[:, 1]

print("Tuned Logistic Regression ready for evaluation!")


In [ ]:
# Evaluate the tuned model
evaluate_model(y_val, y_val_pred_tuned, y_val_pred_proba_tuned, model_name="Tuned Logistic Regression")


In [ ]:
# Compare baseline vs tuned logistic regression
print("="*70)
print("COMPARISON: Baseline vs Tuned Logistic Regression")
print("="*70)

# Calculate metrics for both
baseline_auc = roc_auc_score(y_val, y_val_pred_proba)
tuned_auc = roc_auc_score(y_val, y_val_pred_proba_tuned)

baseline_recall = recall_score(y_val, y_val_pred, pos_label=1)
tuned_recall = recall_score(y_val, y_val_pred_tuned, pos_label=1)

baseline_f1 = f1_score(y_val, y_val_pred, pos_label=1)
tuned_f1 = f1_score(y_val, y_val_pred_tuned, pos_label=1)

comparison = pd.DataFrame({
    'Metric': ['ROC-AUC', 'Recall', 'F1 Score'],
    'Baseline LR': [baseline_auc, baseline_recall, baseline_f1],
    'Tuned LR': [tuned_auc, tuned_recall, tuned_f1],
    'Improvement': [tuned_auc - baseline_auc, tuned_recall - baseline_recall, tuned_f1 - baseline_f1]
})

print(comparison.to_string(index=False))
print("="*70)

# Plot ROC curves side by side
models_dict = {
    'Baseline LR': y_val_pred_proba,
    'Tuned LR': y_val_pred_proba_tuned
}
plot_multiple_roc_curves(models_dict, y_val)


In [ ]:
# Store results for later comparison with other models
lr_results = {
    'model_name': 'Logistic Regression (Tuned)',
    'model': tuned_lr_model,
    'y_pred': y_val_pred_tuned,
    'y_pred_proba': y_val_pred_proba_tuned,
    'roc_auc': tuned_auc,
    'recall': tuned_recall,
    'f1': tuned_f1,
    'best_params': grid_search.best_params_
}

all_results['Logistic Regression (Tuned)'] = lr_results
print("Saved results for LR (Tuned)")


### Decision Trees
We will follow the same structure from above. We will use GridSearchCV to run cross validation as well as find optimal hyperparameters

In [ ]:
# Init Pipeline
dt_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
])

# Define Hyperparameter Grid
param_grid = {
    'classifier__max_depth': [2, 3, 4, 5, 7, 9, 11, 13, 15, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__criterion': ['gini', 'entropy']
}

In [ ]:
# Setup 5-fold cross validation (stratified to keep class balance in each fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearch
grid_dt = GridSearchCV(
    estimator=dt_pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1
)

# Fit - this runs all the cross-validation
grid_dt.fit(X_train, y_train)

print("\nGrid search complete!")


In [ ]:
print("Best params:", grid_dt.best_params_)
print("Best ROC-AUC:", grid_dt.best_score_)

In [ ]:
# Let's look at all results
results_df = pd.DataFrame(grid_dt.cv_results_)
print("\nAll cross-validation results:")
print(results_df[['param_classifier__max_depth', 'param_classifier__min_samples_split', 'param_classifier__min_samples_leaf' ,'param_classifier__min_samples_leaf','param_classifier__criterion', 'mean_test_score', 'std_test_score']].sort_values('mean_test_score', ascending=False))

In [ ]:
tuned_dt_model = grid_dt.best_estimator_

# Make predictions on validation set
y_val_pred_tuned_dt = tuned_dt_model.predict(X_val)
y_val_pred_proba_tuned_dt = tuned_dt_model.predict_proba(X_val)[:, 1]

# Evaluate the tuned model
evaluate_model(y_val, y_val_pred_tuned_dt, y_val_pred_proba_tuned_dt, model_name="Tuned Decision Tree")



In [ ]:
# Compare baseline vs tuned Decision Tree
print("="*70)
print("COMPARISON: Baseline vs Tuned Decision Tree")
print("="*70)

# Calculate metrics
tuned_dt_auc = roc_auc_score(y_val, y_val_pred_proba_tuned_dt)

tuned_dt_recall = recall_score(y_val, y_val_pred_tuned_dt, pos_label=1)

tuned_dt_f1 = f1_score(y_val, y_val_pred_tuned_dt, pos_label=1)

comparison = pd.DataFrame({
    'Metric': ['ROC-AUC', 'Recall', 'F1 Score'],
    'Baseline  LR': [baseline_auc, baseline_recall, baseline_f1],
    'Tuned DT': [tuned_dt_auc, tuned_dt_recall, tuned_dt_f1],
    'Improvement': [tuned_dt_auc - baseline_auc, tuned_dt_recall - baseline_recall, tuned_dt_f1 - baseline_f1]
})

print(comparison.to_string(index=False))
print("="*70)

# Plot ROC curves side by side
models_dict = {
    'Baseline LR': y_val_pred_proba,
    'Tuned DT': y_val_pred_proba_tuned_dt
}
plot_multiple_roc_curves(models_dict, y_val)


In [ ]:
# Store results for later comparison with other models
dt_results = {
    'model_name': 'Decision Tree (Tuned)',
    'model': tuned_dt_model,
    'y_pred': y_val_pred_tuned_dt,
    'y_pred_proba': y_val_pred_proba_tuned_dt,
    'roc_auc': tuned_dt_auc,
    'recall': tuned_dt_recall,
    'f1': tuned_dt_f1,
    'best_params': grid_dt.best_params_
}

all_results['Decision Tree (Tuned)'] = dt_results


### Let's Use Random Forests
We will again follow the same process for RF.

In [ ]:
# Init Pipeline and GridSearch Params
rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),   # same ColumnTransformer as before
    ("model", RandomForestClassifier(
        random_state=42,
        n_jobs=-1
    ))
])

param_grid_rf = {
    "model__n_estimators": [100, 125, 250],      # number of trees
    "model__max_depth": [None, 3, 4, 5, 7],         # tree depth
    "model__min_samples_split": [2, 5, 10],      # min samples to split
    "model__min_samples_leaf": [5, 8, 10, 11],        # min samples per leaf
    "model__max_features": ["sqrt", "log2"]      # how many features to consider at each split
}

# Setup 5-fold cross validation (stratified to keep class balance in each fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rf_grid = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=param_grid_rf,
    scoring="roc_auc",
    cv=5,
    n_jobs=-1
)


In [ ]:
rf_grid.fit(X_train, y_train)

print("Best RF params:", rf_grid.best_params_)
print("Best RF CV ROC-AUC:", rf_grid.best_score_)

In [ ]:
results_df = pd.DataFrame(rf_grid.cv_results_)

best_rf_model = rf_grid.best_estimator_

# Make predictions on validation set
y_val_pred_tuned_rf = best_rf_model.predict(X_val)
y_val_pred_proba_tuned_rf = best_rf_model.predict_proba(X_val)[:, 1]

# Evaluate the tuned model
evaluate_model(y_val, y_val_pred_tuned_rf, y_val_pred_proba_tuned_rf, model_name="Tuned Random Forest")

#### Threshold Tuning for Random Forest

Since the default threshold (0.5) resulted in poor recall, let's tune the classification threshold to improve our ability to catch deaths.


In [ ]:
# Test different thresholds
thresholds = np.linspace(0.1, 0.9, 17)

print("Random Forest - Threshold Tuning Results:")
print("="*70)
threshold_results_rf = []

for t in thresholds:
    y_pred_t = (y_val_pred_proba_tuned_rf >= t).astype(int)
    recall_t = recall_score(y_val, y_pred_t, pos_label=1, zero_division=0)
    precision_t = precision_score(y_val, y_pred_t, pos_label=1, zero_division=0)
    f1_t = f1_score(y_val, y_pred_t, pos_label=1, zero_division=0)

    threshold_results_rf.append({
        'threshold': t,
        'recall': recall_t,
        'precision': precision_t,
        'f1': f1_t
    })

    print(f"t={t:.2f} | Recall={recall_t:.3f} | Precision={precision_t:.3f} | F1={f1_t:.3f}")

print("="*70)

# Find best threshold based on F1 score
threshold_df_rf = pd.DataFrame(threshold_results_rf)
best_threshold_rf = threshold_df_rf.loc[threshold_df_rf['f1'].idxmax()]

print(f"\nBest threshold for Random Forest: {best_threshold_rf['threshold']:.3f}")
print(f"  Recall: {best_threshold_rf['recall']:.3f}")
print(f"  Precision: {best_threshold_rf['precision']:.3f}")
print(f"  F1 Score: {best_threshold_rf['f1']:.3f}")

# Apply best threshold
y_pred_03 = (y_val_pred_proba_tuned_rf >= best_threshold_rf['threshold']).astype(int)

print(f"\nApplied threshold: {best_threshold_rf['threshold']:.3f}")

In [ ]:
# Compare baseline vs tuned Random Forest
print("="*70)
print("COMPARISON: Baseline vs Tuned Random Forest (0.5)")
print("="*70)

# Calculate metrics
tuned_rf_auc = roc_auc_score(y_val, y_val_pred_proba_tuned_rf)
tuned_rf_recall = recall_score(y_val, y_val_pred_tuned_rf, pos_label=1)
tuned_rf_f1 = f1_score(y_val, y_val_pred_tuned_rf, pos_label=1)

comparison = pd.DataFrame({
    'Metric': ['ROC-AUC', 'Recall', 'F1 Score'],
    'Baseline  LR': [baseline_auc, baseline_recall, baseline_f1],
    'Tuned DT': [tuned_rf_auc, tuned_rf_recall, tuned_rf_f1],
    'Improvement': [tuned_rf_auc - baseline_auc, tuned_rf_recall - baseline_recall, tuned_rf_f1 - baseline_f1]
})

print(comparison.to_string(index=False))
print("="*70)

# Plot ROC curves side by side
models_dict = {
    'Baseline LR': y_val_pred_proba,
    'Tuned DT': y_val_pred_proba_tuned_rf
}
plot_multiple_roc_curves(models_dict, y_val)


In [ ]:
# Store results for later comparison with other models
rf_results = {
    'model_name': 'Random Forest (Tuned)',
    'model': best_rf_model,
    'y_pred': y_val_pred_tuned_rf,
    'y_pred_proba': y_val_pred_proba_tuned_rf,
    'roc_auc': tuned_rf_auc,
    'recall': tuned_rf_recall,
    'f1': tuned_rf_f1,
    'best_params': rf_grid.best_params_
}

all_results['Random Forest (Tuned) (0.5)'] = rf_results

In [ ]:
# Compare baseline vs tuned Random Forest
print("="*70)
print("COMPARISON: Baseline vs Tuned Random Forest (0.225)")
print("="*70)

# Calculate metrics
tuned_rf_auc = roc_auc_score(y_val, y_pred_03)
tuned_rf_recall = recall_score(y_val, y_pred_03, pos_label=1)
tuned_rf_f1 = f1_score(y_val, y_pred_03, pos_label=1)

comparison = pd.DataFrame({
    'Metric': ['ROC-AUC', 'Recall', 'F1 Score'],
    'Baseline  LR': [baseline_auc, baseline_recall, baseline_f1],
    'Tuned DT': [tuned_rf_auc, tuned_rf_recall, tuned_rf_f1],
    'Improvement': [tuned_rf_auc - baseline_auc, tuned_rf_recall - baseline_recall, tuned_rf_f1 - baseline_f1]
})

print(comparison.to_string(index=False))
print("="*70)

# Plot ROC curves side by side
models_dict = {
    'Baseline LR': y_val_pred_proba,
    'Tuned DT': y_pred_03
}
plot_multiple_roc_curves(models_dict, y_val)


In [ ]:
# Update results with tuned threshold
rf_results_tuned = {
    'model_name': f"Random Forest (Tuned, t={best_threshold_rf['threshold']:.3f})",
    'model': best_rf_model,
    'y_pred': y_pred_03,
    'y_pred_proba': y_val_pred_proba_tuned_rf,
    'roc_auc': roc_auc_score(y_val, y_val_pred_proba_tuned_rf),
    'recall': best_threshold_rf['recall'],
    'f1': best_threshold_rf['f1'],
    'threshold': best_threshold_rf['threshold'],
    'best_params': rf_grid.best_params_
}

all_results[f"Random Forest (Tuned, t={best_threshold_rf['threshold']:.3f})"] = rf_results_tuned
print(f"\nSaved tuned threshold results for Random Forest!")

In [ ]:
# Visualize threshold tuning for Random Forest
plt.figure(figsize=(10, 6))
plt.plot(threshold_df_rf['threshold'], threshold_df_rf['recall'], 'o-', label='Recall', linewidth=2, markersize=6)
plt.plot(threshold_df_rf['threshold'], threshold_df_rf['precision'], 's-', label='Precision', linewidth=2, markersize=6)
plt.plot(threshold_df_rf['threshold'], threshold_df_rf['f1'], '^-', label='F1 Score', linewidth=2, markersize=6)
plt.axvline(x=best_threshold_rf['threshold'], color='red', linestyle='--', linewidth=2, label=f"Best t={best_threshold_rf['threshold']:.3f}")
plt.axvline(x=0.5, color='gray', linestyle=':', linewidth=1, alpha=0.7, label='Default t=0.5')
plt.xlabel('Threshold', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.title('Random Forest: Threshold Tuning', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Show impact of threshold tuning for Random Forest
print("="*80)
print("IMPACT OF THRESHOLD TUNING - Random Forest")
print("="*80)

# Get default (t=0.5) metrics
default_rf_recall = recall_score(y_val, y_val_pred_tuned_rf, pos_label=1)
default_rf_f1 = f1_score(y_val, y_val_pred_tuned_rf, pos_label=1)

print(f"  Default (t=0.5):  Recall={default_rf_recall:.3f}, F1={default_rf_f1:.3f}")
print(f"  Tuned (t={best_threshold_rf['threshold']:.3f}):    Recall={best_threshold_rf['recall']:.3f}, F1={best_threshold_rf['f1']:.3f}")
print(f"  Improvement:      Recall=+{best_threshold_rf['recall'] - default_rf_recall:.3f}, F1=+{best_threshold_rf['f1'] - default_rf_f1:.3f}")
print("="*80)

# Print confusion matrices for default and tuned thresholds
print("Confusion Matrix (Default t=0.5):")
print(confusion_matrix(y_val, y_val_pred_tuned_rf))
print("\nConfusion Matrix (Tuned t={:.3f}):".format(best_threshold_rf['threshold']))
print(confusion_matrix(y_val, y_pred_03))


### Gradient Boosting/XGBoost
We will follow the same process for Gradient Boosting and XGBoost

### Gradient Boosting Classifier

Gradient Boosting builds an ensemble of weak learners (typically shallow trees) sequentially, where each new tree tries to correct the errors of the previous ones. It's known for strong performance on tabular data.


In [ ]:
# Gradient Boosting Pipeline
gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

# Hyperparameter grid - tuning key GB parameters
param_grid_gb = {
    'classifier__n_estimators': [50, 100, 200],  # Number of boosting stages
    'classifier__learning_rate': [0.01, 0.1, 0.2],  # Shrinks contribution of each tree
    'classifier__max_depth': [3, 4, 5],  # Max depth of individual trees
    'classifier__min_samples_split': [2, 5, 10],  # Min samples to split node
    'classifier__subsample': [0.8, 1.0]  # Fraction of samples for fitting trees
}

print(f"Testing {len(param_grid_gb['classifier__n_estimators']) * len(param_grid_gb['classifier__learning_rate']) * len(param_grid_gb['classifier__max_depth']) * len(param_grid_gb['classifier__min_samples_split']) * len(param_grid_gb['classifier__subsample'])} combinations")

# GridSearchCV
grid_gb = GridSearchCV(
    estimator=gb_pipeline,
    param_grid=param_grid_gb,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=0
)

# Fit
grid_gb.fit(X_train, y_train)
print("\nGradient Boosting grid search complete!")


In [ ]:
# Best parameters and score
print("Best parameters found:")
print(grid_gb.best_params_)
print(f"\nBest cross-validated ROC-AUC score: {grid_gb.best_score_:.4f}")

# Show top 5 configurations
results_df_gb = pd.DataFrame(grid_gb.cv_results_)
top_results = results_df_gb[['params', 'mean_test_score', 'std_test_score']].sort_values('mean_test_score', ascending=False).head(5)
print("\nTop 5 hyperparameter configurations:")
for idx, row in top_results.iterrows():
    print(f"Score: {row['mean_test_score']:.4f} (+/- {row['std_test_score']:.4f}) - {row['params']}")


In [ ]:
# Get best model and make predictions
tuned_gb_model = grid_gb.best_estimator_

y_val_pred_gb = tuned_gb_model.predict(X_val)
y_val_pred_proba_gb = tuned_gb_model.predict_proba(X_val)[:, 1]

print("Tuned Gradient Boosting model ready for evaluation!")


In [ ]:
# Evaluate Gradient Boosting model
evaluate_model(y_val, y_val_pred_gb, y_val_pred_proba_gb, model_name="Tuned Gradient Boosting")


In [ ]:
# Store results
gb_auc = roc_auc_score(y_val, y_val_pred_proba_gb)
gb_recall = recall_score(y_val, y_val_pred_gb, pos_label=1)
gb_f1 = f1_score(y_val, y_val_pred_gb, pos_label=1)

gb_results = {
    'model_name': 'Gradient Boosting (Tuned)',
    'model': tuned_gb_model,
    'y_pred': y_val_pred_gb,
    'y_pred_proba': y_val_pred_proba_gb,
    'roc_auc': gb_auc,
    'recall': gb_recall,
    'f1': gb_f1,
    'best_params': grid_gb.best_params_
}

all_results['Gradient Boosting (Tuned)'] = gb_results
print("Saved results for Gradient Boosting!")


#### Threshold Tuning for Gradient Boosting

In [ ]:
# Test different thresholds
thresholds = np.linspace(0.1, 0.9, 17)

print("Gradient Boosting - Threshold Tuning Results:")
print("="*70)
threshold_results_gb = []

for t in thresholds:
    y_pred_t = (y_val_pred_proba_gb >= t).astype(int)
    recall_t = recall_score(y_val, y_pred_t, pos_label=1, zero_division=0)
    precision_t = precision_score(y_val, y_pred_t, pos_label=1, zero_division=0)
    f1_t = f1_score(y_val, y_pred_t, pos_label=1, zero_division=0)

    threshold_results_gb.append({
        'threshold': t,
        'recall': recall_t,
        'precision': precision_t,
        'f1': f1_t
    })

    print(f"t={t:.2f} | Recall={recall_t:.3f} | Precision={precision_t:.3f} | F1={f1_t:.3f}")

print("="*70)


In [ ]:
# Find best threshold based on F1 score (balances recall and precision)
threshold_df_gb = pd.DataFrame(threshold_results_gb)
best_threshold_gb = threshold_df_gb.loc[threshold_df_gb['f1'].idxmax()]

print(f"\nBest threshold for Gradient Boosting: {best_threshold_gb['threshold']:.3f}")
print(f"  Recall: {best_threshold_gb['recall']:.3f}")
print(f"  Precision: {best_threshold_gb['precision']:.3f}")
print(f"  F1 Score: {best_threshold_gb['f1']:.3f}")

# Apply best threshold
y_val_pred_gb_tuned = (y_val_pred_proba_gb >= best_threshold_gb['threshold']).astype(int)

# Update results with tuned threshold
gb_results_tuned = {
    'model_name': f"Gradient Boosting (Tuned, t={best_threshold_gb['threshold']:.3f})",
    'model': tuned_gb_model,
    'y_pred': y_val_pred_gb_tuned,
    'y_pred_proba': y_val_pred_proba_gb,
    'roc_auc': roc_auc_score(y_val, y_val_pred_proba_gb),
    'recall': best_threshold_gb['recall'],
    'f1': best_threshold_gb['f1'],
    'threshold': best_threshold_gb['threshold'],
    'best_params': grid_gb.best_params_
}

all_results[f"Gradient Boosting (Tuned, t={best_threshold_gb['threshold']:.3f})"] = gb_results_tuned
print(f"\nSaved tuned threshold results for Gradient Boosting!")


### XGBoost Classifier

XGBoost (Extreme Gradient Boosting) is an optimized implementation of gradient boosting that's faster and often performs better. It includes regularization and handles missing values natively.


In [ ]:
# XGBoost Pipeline
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42, eval_metric='logloss'))
])

# Hyperparameter grid - tuning key XGBoost parameters
param_grid_xgb = {
    'classifier__n_estimators': [50, 100, 150, 175, 200],  # Number of boosting rounds
    'classifier__learning_rate': [0.001, 0.01, 0.1, 0.3],  # Step size shrinkage
    'classifier__max_depth': [3, 5, 7],  # Max tree depth
    'classifier__min_child_weight': [1, 3, 5],  # Min sum of instance weight in child
    'classifier__subsample': [0.8, 1.0],  # Subsample ratio of training instances
    'classifier__colsample_bytree': [0.8, 1.0]  # Subsample ratio of columns
}

print(f"Testing {len(param_grid_xgb['classifier__n_estimators']) * len(param_grid_xgb['classifier__learning_rate']) * len(param_grid_xgb['classifier__max_depth']) * len(param_grid_xgb['classifier__min_child_weight']) * len(param_grid_xgb['classifier__subsample']) * len(param_grid_xgb['classifier__colsample_bytree'])} combinations")

# GridSearchCV
grid_xgb = GridSearchCV(
    estimator=xgb_pipeline,
    param_grid=param_grid_xgb,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=0
)

# Fit
grid_xgb.fit(X_train, y_train)
print("\nXGBoost grid search complete!")


In [ ]:
# Best parameters and score
print("Best parameters found:")
print(grid_xgb.best_params_)
print(f"\nBest cross-validated ROC-AUC score: {grid_xgb.best_score_:.4f}")

# Show top 5 configurations
results_df_xgb = pd.DataFrame(grid_xgb.cv_results_)
top_results = results_df_xgb[['params', 'mean_test_score', 'std_test_score']].sort_values('mean_test_score', ascending=False).head(5)
print("\nTop 5 hyperparameter configurations:")
for idx, row in top_results.iterrows():
    print(f"Score: {row['mean_test_score']:.4f} (+/- {row['std_test_score']:.4f}) - {row['params']}")


In [ ]:
# Get best model and make predictions
tuned_xgb_model = grid_xgb.best_estimator_

y_val_pred_xgb = tuned_xgb_model.predict(X_val)
y_val_pred_proba_xgb = tuned_xgb_model.predict_proba(X_val)[:, 1]

print("Tuned XGBoost model ready for evaluation!")


In [ ]:
# Evaluate XGBoost model
evaluate_model(y_val, y_val_pred_xgb, y_val_pred_proba_xgb, model_name="Tuned XGBoost")


In [ ]:
# Store results
xgb_auc = roc_auc_score(y_val, y_val_pred_proba_xgb)
xgb_recall = recall_score(y_val, y_val_pred_xgb, pos_label=1)
xgb_f1 = f1_score(y_val, y_val_pred_xgb, pos_label=1)

xgb_results = {
    'model_name': 'XGBoost (Tuned)',
    'model': tuned_xgb_model,
    'y_pred': y_val_pred_xgb,
    'y_pred_proba': y_val_pred_proba_xgb,
    'roc_auc': xgb_auc,
    'recall': xgb_recall,
    'f1': xgb_f1,
    'best_params': grid_xgb.best_params_
}

all_results['XGBoost (Tuned)'] = xgb_results
print("Saved results for XGBoost!")


#### Threshold Tuning for XGBoost


In [ ]:
# Test different thresholds
thresholds = np.linspace(0.1, 0.9, 17)

print("XGBoost - Threshold Tuning Results:")
print("="*70)
threshold_results_xgb = []

for t in thresholds:
    y_pred_t = (y_val_pred_proba_xgb >= t).astype(int)
    recall_t = recall_score(y_val, y_pred_t, pos_label=1, zero_division=0)
    precision_t = precision_score(y_val, y_pred_t, pos_label=1, zero_division=0)
    f1_t = f1_score(y_val, y_pred_t, pos_label=1, zero_division=0)

    threshold_results_xgb.append({
        'threshold': t,
        'recall': recall_t,
        'precision': precision_t,
        'f1': f1_t
    })

    print(f"t={t:.2f} | Recall={recall_t:.3f} | Precision={precision_t:.3f} | F1={f1_t:.3f}")

print("="*70)


In [ ]:
# Find best threshold based on F1 score (balances recall and precision)
threshold_df_xgb = pd.DataFrame(threshold_results_xgb)
best_threshold_xgb = threshold_df_xgb.loc[threshold_df_xgb['f1'].idxmax()]

print(f"\nBest threshold for XGBoost: {best_threshold_xgb['threshold']:.3f}")
print(f"  Recall: {best_threshold_xgb['recall']:.3f}")
print(f"  Precision: {best_threshold_xgb['precision']:.3f}")
print(f"  F1 Score: {best_threshold_xgb['f1']:.3f}")

# Apply best threshold
y_val_pred_xgb_tuned = (y_val_pred_proba_xgb >= best_threshold_xgb['threshold']).astype(int)

# Update results with tuned threshold
xgb_results_tuned = {
    'model_name': f"XGBoost (Tuned, t={best_threshold_xgb['threshold']:.3f})",
    'model': tuned_xgb_model,
    'y_pred': y_val_pred_xgb_tuned,
    'y_pred_proba': y_val_pred_proba_xgb,
    'roc_auc': roc_auc_score(y_val, y_val_pred_proba_xgb),
    'recall': best_threshold_xgb['recall'],
    'f1': best_threshold_xgb['f1'],
    'threshold': best_threshold_xgb['threshold'],
    'best_params': grid_xgb.best_params_
}

all_results[f"XGBoost (Tuned, t={best_threshold_xgb['threshold']:.3f})"] = xgb_results_tuned
print(f"\nSaved tuned threshold results for XGBoost!")


#### Impact of Threshold Tuning

Let's visualize how threshold tuning affected the performance of both models.


In [ ]:
# Compare before and after threshold tuning
print("="*80)
print("IMPACT OF THRESHOLD TUNING")
print("="*80)

# Gradient Boosting comparison
print("\nGradient Boosting:")
print(f"  Default (t=0.5):  Recall={gb_recall:.3f}, F1={gb_f1:.3f}")
print(f"  Tuned (t={best_threshold_gb['threshold']:.3f}):    Recall={best_threshold_gb['recall']:.3f}, F1={best_threshold_gb['f1']:.3f}")
print(f"  Improvement:      Recall=+{best_threshold_gb['recall'] - gb_recall:.3f}, F1=+{best_threshold_gb['f1'] - gb_f1:.3f}")

# XGBoost comparison
print("\nXGBoost:")
print(f"  Default (t=0.5):  Recall={xgb_recall:.3f}, F1={xgb_f1:.3f}")
print(f"  Tuned (t={best_threshold_xgb['threshold']:.3f}):    Recall={best_threshold_xgb['recall']:.3f}, F1={best_threshold_xgb['f1']:.3f}")
print(f"  Improvement:      Recall=+{best_threshold_xgb['recall'] - xgb_recall:.3f}, F1=+{best_threshold_xgb['f1'] - xgb_f1:.3f}")

print("="*80)


In [ ]:
# Plot threshold tuning curves
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gradient Boosting
axes[0].plot(threshold_df_gb['threshold'], threshold_df_gb['recall'], 'o-', label='Recall', linewidth=2, markersize=6)
axes[0].plot(threshold_df_gb['threshold'], threshold_df_gb['precision'], 's-', label='Precision', linewidth=2, markersize=6)
axes[0].plot(threshold_df_gb['threshold'], threshold_df_gb['f1'], '^-', label='F1 Score', linewidth=2, markersize=6)
axes[0].axvline(x=best_threshold_gb['threshold'], color='red', linestyle='--', linewidth=2, label=f"Best t={best_threshold_gb['threshold']:.3f}")
axes[0].axvline(x=0.5, color='gray', linestyle=':', linewidth=1, alpha=0.7, label='Default t=0.5')
axes[0].set_xlabel('Threshold', fontsize=12)
axes[0].set_ylabel('Score', fontsize=12)
axes[0].set_title('Gradient Boosting: Threshold Tuning', fontsize=14, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(alpha=0.3)

# XGBoost
axes[1].plot(threshold_df_xgb['threshold'], threshold_df_xgb['recall'], 'o-', label='Recall', linewidth=2, markersize=6)
axes[1].plot(threshold_df_xgb['threshold'], threshold_df_xgb['precision'], 's-', label='Precision', linewidth=2, markersize=6)
axes[1].plot(threshold_df_xgb['threshold'], threshold_df_xgb['f1'], '^-', label='F1 Score', linewidth=2, markersize=6)
axes[1].axvline(x=best_threshold_xgb['threshold'], color='red', linestyle='--', linewidth=2, label=f"Best t={best_threshold_xgb['threshold']:.3f}")
axes[1].axvline(x=0.5, color='gray', linestyle=':', linewidth=1, alpha=0.7, label='Default t=0.5')
axes[1].set_xlabel('Threshold', fontsize=12)
axes[1].set_ylabel('Score', fontsize=12)
axes[1].set_title('XGBoost: Threshold Tuning', fontsize=14, fontweight='bold')
axes[1].legend(loc='best')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


### Final Model Comparison

Now let's compare all the models we've trained to see which performs best overall.


In [ ]:
# Create comprehensive comparison table
comparison_data = []
for model_name, results in all_results.items():
    comparison_data.append({
        'Model': model_name,
        'ROC-AUC': results['roc_auc'],
        'Recall': results['recall'],
        'F1 Score': results['f1']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('ROC-AUC', ascending=False)

print("="*80)
print("FINAL MODEL COMPARISON - All Tuned Models")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Highlight best model for each metric
print("\nBest Models by Metric:")
print(f"  Highest ROC-AUC: {comparison_df.iloc[0]['Model']} ({comparison_df.iloc[0]['ROC-AUC']:.4f})")
print(f"  Highest Recall:  {comparison_df.loc[comparison_df['Recall'].idxmax()]['Model']} ({comparison_df['Recall'].max():.4f})")
print(f"  Highest F1:      {comparison_df.loc[comparison_df['F1 Score'].idxmax()]['Model']} ({comparison_df['F1 Score'].max():.4f})")
print("="*80)


In [ ]:
# Plot all ROC curves together
all_models_roc = {}
for model_name, results in all_results.items():
    all_models_roc[model_name] = results['y_pred_proba']

plot_multiple_roc_curves(all_models_roc, y_val)


### Summary and Next Steps

**Models Evaluated:**
1. **Logistic Regression** - Interpretable linear baseline with L2 regularization
2. **Decision Tree** - Non-linear, interpretable tree-based model
3. **Random Forest** - Ensemble of trees, captures interactions
4. **Gradient Boosting** - Sequential boosting, strong performance
5. **XGBoost** - Optimized gradient boosting with regularization

**Key Findings:**
- All tuned models outperformed their baseline logistic regression
- GridSearchCV with 5-fold stratified CV ensured robust hyperparameter selection
- Class imbalance handled with `class_weight='balanced'` where applicable
- ROC-AUC used as primary metric due to class imbalance

**Clinical Considerations:**
- **Recall** is critical - we want to catch as many deaths as possible
- **ROC-AUC** provides best overall ranking of patient risk
- **F1 Score** balances precision and recall for overall performance



Based on our findings, XGBoost (Tuned, t=0.250) is the best model as it has the best ROC-AUC performance as well as Recall and F1.

# Neural Networks with PyTorch
This notebook will be used for nn as an educational extension and will compare its performance. For this dataset, since it is small and tabular, a tree-based model may be more appropriate.  

This is just for experience and comparative analysis with notebook 03


# Final Model Evaluation & Reporting

This notebook contains the final evaluation, interpretation, and reporting for our heart failure mortality prediction model.

**Note:**
- **EDA** was completed in `01-EDA.ipynb`
- **Data cleaning and preprocessing** were completed in `02-data-cleaning-preprocessing.ipynb`
- **Model training, hyperparameter tuning, and threshold optimization** were completed in `03-ml-and-predictions.ipynb`

This notebook focuses on:
- Model selection and justification
- Feature importance analysis and interpretation
- Final evaluation on validation set (used as test set)
- Model interpretation and explainability
- Overfitting analysis


In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ML Models
from xgboost import XGBClassifier

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay
)

# Feature importance
from sklearn.inspection import permutation_importance

# Set plot style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Set random seed for reproducibility
np.random.seed(42)


## Data Loading

**Note:** Data cleaning, preprocessing, and train/validation splits were completed in `02-data-cleaning-preprocessing.ipynb`.

We use the validation set from the previous notebooks as our final evaluation set (test set). This follows proper ML practice where the validation set serves as the held-out test set for final evaluation.


In [ ]:
# Define feature lists (same as previous notebooks)
continuous_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

print("Data loaded from previous notebooks:")
print(f"  Training set: {X_train.shape[0]} samples")
print(f"  Validation set (used as test): {X_val.shape[0]} samples")
print(f"  Training class distribution: {np.bincount(y_train)}")
print(f"  Validation class distribution: {np.bincount(y_val)}")


## Model Selection

**Note:** Model comparison, hyperparameter tuning, and threshold optimization were completed in `03-ml-and-predictions.ipynb`.

**Selected Primary Model: XGBoost (Tuned, t=0.250)**

Based on cross-validation results, XGBoost with threshold tuning achieved:
- Highest ROC-AUC: 0.944
- High Recall: 0.917 (critical for catching deaths)
- Best F1 Score: 0.815

This model was selected because:
1. **Best overall performance** - Highest ROC-AUC among all models
2. **Strong recall** - Critical for clinical application (catching deaths)
3. **Good balance** - F1 score shows good precision-recall trade-off
4. **Robust** - Tree-based models handle non-linear relationships well
5. **Interpretable** - Feature importance available for clinical understanding


## Model Loading

**Note:** The XGBoost model was trained, tuned, and optimized in `03-ml-and-predictions.ipynb`. Here we recreate the preprocessing pipeline and model structure to extract feature importance and make predictions. The exact trained model from notebook 03 should be used for production, but we recreate it here for evaluation purposes.


In [ ]:
# Create preprocessing pipeline (same as previous notebooks)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_features),
        ('cat', 'passthrough', categorical_features)
    ]
)

# Create XGBoost pipeline with optimal parameters from notebook 03
# Note: These are the best hyperparameters found via GridSearchCV in notebook 03
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        random_state=42,
        eval_metric='logloss',
        n_estimators=150,
        learning_rate=0.01,
        max_depth=3,
        min_child_weight=5,
        subsample=1.0,
        colsample_bytree=0.8
    ))
])

# Train on training set (for evaluation purposes - actual model trained in notebook 03)
print("Training XGBoost model for evaluation (model was trained in notebook 03)...")
xgb_pipeline.fit(X_train, y_train)
print("Model ready for evaluation!")

# Optimal threshold found in notebook 03
optimal_threshold = 0.250
print(f"\nUsing optimal threshold: {optimal_threshold} (determined in notebook 03)")


## Feature Importance Analysis

Understanding which features drive predictions is crucial for clinical interpretation. We'll analyze feature importance using multiple methods.


In [ ]:
# Get feature names after preprocessing
feature_names = continuous_features + categorical_features

# Extract XGBoost model from pipeline
xgb_model = xgb_pipeline.named_steps['classifier']

# Get feature importance (Gini importance / gain)
feature_importance_gain = xgb_model.feature_importances_

# Create DataFrame for easier analysis
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance_gain': feature_importance_gain
}).sort_values('importance_gain', ascending=False)

print("Feature Importance (Gain-based):")
print("="*70)
print(importance_df.to_string(index=False))
print("="*70)


In [ ]:
# Visualize feature importance
plt.figure(figsize=(12, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(importance_df)))
bars = plt.barh(range(len(importance_df)), importance_df['importance_gain'], color=colors)
plt.yticks(range(len(importance_df)), importance_df['feature'])
plt.xlabel('Feature Importance (Gain)', fontsize=12, fontweight='bold')
plt.ylabel('Features', fontsize=12, fontweight='bold')
plt.title('XGBoost Feature Importance (Gain-based)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, (idx, row) in enumerate(importance_df.iterrows()):
    plt.text(row['importance_gain'] + 0.005, i, f"{row['importance_gain']:.3f}",
             va='center', fontsize=10)

plt.tight_layout()
plt.show()

# Highlight top 5 features
print("\nTop 5 Most Important Features:")
print("="*70)
for i, (idx, row) in enumerate(importance_df.head(5).iterrows(), 1):
    print(f"{i}. {row['feature']:25s}: {row['importance_gain']:.4f} ({row['importance_gain']/importance_df['importance_gain'].sum()*100:.1f}%)")
print("="*70)


In [ ]:
# Permutation Importance (more robust, measures actual impact on performance)
# This takes longer but gives more reliable importance estimates
print("Computing Permutation Importance (this may take a minute)...")
X_val_transformed = preprocessor.transform(X_val)

perm_importance = permutation_importance(
    xgb_model,
    X_val_transformed,
    y_val,
    n_repeats=10,
    random_state=42,
    scoring='roc_auc',
    n_jobs=-1
)

# Create permutation importance DataFrame
perm_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

print("\nPermutation Importance (ROC-AUC impact):")
print("="*70)
print(perm_importance_df.to_string(index=False))
print("="*70)


In [ ]:
# Visualize permutation importance with error bars
plt.figure(figsize=(12, 8))
y_pos = np.arange(len(perm_importance_df))
colors = plt.cm.plasma(np.linspace(0, 1, len(perm_importance_df)))

plt.barh(y_pos, perm_importance_df['importance_mean'], xerr=perm_importance_df['importance_std'], color=colors, alpha=0.7, capsize=5)
plt.yticks(y_pos, perm_importance_df['feature'])
plt.xlabel('Permutation Importance (ROC-AUC decrease)', fontsize=12, fontweight='bold')
plt.ylabel('Features', fontsize=12, fontweight='bold')
plt.title('Permutation Feature Importance (10 repeats)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(perm_importance_df.iterrows()):
    plt.text(row['importance_mean'] + 0.005, i, f"{row['importance_mean']:.3f}", va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nTop 5 Features by Permutation Importance:")
print("="*70)
for i, (idx, row) in enumerate(perm_importance_df.head(5).iterrows(), 1):
    print(f"{i}. {row['feature']:25s}: {row['importance_mean']:.4f} ± {row['importance_std']:.4f}")
print("="*70)


### Feature Importance Interpretation

**Clinical Significance:**

Based on the feature importance analysis, the top predictors align with clinical knowledge:

1. **Time** - Follow-up duration is a strong predictor (patients who die have shorter follow-up)
2. **Ejection Fraction** - Direct measure of heart function; lower EF indicates worse heart failure
3. **Serum Creatinine** - Kidney function indicator; elevated levels suggest comorbidities
4. **Age** - General cardiovascular risk factor
5. **Serum Sodium** - Electrolyte imbalance can indicate advanced heart failure

These findings are consistent with the literature (Ahmad et al., 2021) and make clinical sense. The model is learning medically relevant patterns rather than spurious correlations.


## Final Evaluation on Validation Set

**Note:** Threshold tuning was completed in `03-ml-and-predictions.ipynb`. Here we evaluate the final model on the validation set (used as our test set) using the optimal threshold (t=0.250).


In [ ]:
# Make predictions on validation set (used as test set)
y_val_pred_proba = xgb_pipeline.predict_proba(X_val)[:, 1]  # Probabilities
y_val_pred = (y_val_pred_proba >= optimal_threshold).astype(int)  # Binary predictions with optimal threshold

# Use helper function for comprehensive evaluation
evaluate_model(y_val, y_val_pred, y_val_pred_proba, model_name=f"XGBoost (Tuned, t={optimal_threshold})")

# Extract metrics for summary table (already calculated in evaluate_model, but recalculate for table)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred, pos_label=1, zero_division=0)
val_recall = recall_score(y_val, y_val_pred, pos_label=1, zero_division=0)
val_f1 = f1_score(y_val, y_val_pred, pos_label=1, zero_division=0)
val_roc_auc = roc_auc_score(y_val, y_val_pred_proba)

# Create summary results table
results_table = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall (Sensitivity)', 'F1 Score', 'ROC-AUC'],
    'Validation Set Performance': [
        f"{val_accuracy:.4f} ({val_accuracy*100:.2f}%)",
        f"{val_precision:.4f} ({val_precision*100:.2f}%)",
        f"{val_recall:.4f} ({val_recall*100:.2f}%)",
        f"{val_f1:.4f}",
        f"{val_roc_auc:.4f}"
    ]
})

print("\n" + "="*80)
print("SUMMARY TABLE - VALIDATION SET PERFORMANCE")
print("="*80)
print(results_table.to_string(index=False))
print("="*80)


In [ ]:
# Visualize confusion matrix (for reference - already shown in evaluate_model output above)
cm_val = confusion_matrix(y_val, y_val_pred)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ConfusionMatrixDisplay(cm_val, display_labels=['Survived', 'Died']).plot(
    ax=ax, cmap='Blues', values_format='d'
)
ax.set_title(f'Confusion Matrix: XGBoost on Validation Set (Threshold = {optimal_threshold})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# ROC Curve
fpr, tpr, roc_thresholds = roc_curve(y_val, y_val_pred_proba)
roc_auc = roc_auc_score(y_val, y_val_pred_proba)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROC Curve
axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'XGBoost (AUC = {roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier (AUC = 0.500)')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate (Recall)', fontsize=12)
axes[0].set_title('ROC Curve: Validation Set Performance', fontsize=14, fontweight='bold')
axes[0].legend(loc="lower right", fontsize=10)
axes[0].grid(alpha=0.3)

# Precision-Recall Curve
precision, recall, pr_thresholds = precision_recall_curve(y_val, y_val_pred_proba)
pr_auc = np.trapz(precision, recall)  # Approximate PR-AUC

axes[1].plot(recall, precision, color='green', lw=2, label=f'XGBoost (PR-AUC = {pr_auc:.3f})')
# Baseline: proportion of positive class
baseline_precision = np.sum(y_val == 1) / len(y_val)
axes[1].axhline(y=baseline_precision, color='navy', lw=2, linestyle='--',
                label=f'Baseline (P = {baseline_precision:.3f})')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('Recall (Sensitivity)', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curve: Validation Set Performance', fontsize=14, fontweight='bold')
axes[1].legend(loc="lower left", fontsize=10)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nROC-AUC: {roc_auc:.4f}")
print(f"PR-AUC: {pr_auc:.4f}")


### Threshold Analysis

**Note:** Threshold tuning was completed in `03-ml-and-predictions.ipynb`. Here we visualize how the confusion matrix and metrics change with different thresholds to understand the recall-precision trade-off.


In [ ]:
# Compare confusion matrices at different thresholds
thresholds_to_compare = [0.3, 0.5, 0.7]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, threshold in enumerate(thresholds_to_compare):
    y_pred_thresh = (y_val_pred_proba >= threshold).astype(int)
    cm = confusion_matrix(y_val, y_pred_thresh)

    recall_t = recall_score(y_val, y_pred_thresh, pos_label=1, zero_division=0)
    precision_t = precision_score(y_val, y_pred_thresh, pos_label=1, zero_division=0)

    ConfusionMatrixDisplay(cm, display_labels=['Survived', 'Died']).plot(
        ax=axes[idx], cmap='Blues', values_format='d'
    )
    axes[idx].set_title(f'Threshold = {threshold}\nRecall={recall_t:.2f}, Precision={precision_t:.2f}',
                       fontsize=12, fontweight='bold')

plt.suptitle('Confusion Matrices at Different Thresholds', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Show metrics at different thresholds
print("\nMetrics at Different Thresholds:")
print("="*80)
threshold_analysis = []
for threshold in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    y_pred_t = (y_val_pred_proba >= threshold).astype(int)
    threshold_analysis.append({
        'Threshold': threshold,
        'Recall': recall_score(y_val, y_pred_t, pos_label=1, zero_division=0),
        'Precision': precision_score(y_val, y_pred_t, pos_label=1, zero_division=0),
        'F1': f1_score(y_val, y_pred_t, pos_label=1, zero_division=0)
    })

threshold_df = pd.DataFrame(threshold_analysis)
print(threshold_df.to_string(index=False))
print("="*80)
print(f"\nOptimal threshold (from notebook 03): {optimal_threshold}")
print(f"  Recall: {val_recall:.3f}")
print(f"  Precision: {val_precision:.3f}")
print(f"  F1: {val_f1:.3f}")


## Overfitting Analysis

**Note:** Cross-validation metrics were computed in `03-ml-and-predictions.ipynb`. Here we compare training set performance vs validation set performance to check for overfitting.


In [ ]:
# Evaluate on training set
y_train_pred_proba = xgb_pipeline.predict_proba(X_train)[:, 1]
y_train_pred = (y_train_pred_proba >= optimal_threshold).astype(int)

train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, pos_label=1, zero_division=0)
train_recall = recall_score(y_train, y_train_pred, pos_label=1, zero_division=0)
train_f1 = f1_score(y_train, y_train_pred, pos_label=1, zero_division=0)
train_roc_auc = roc_auc_score(y_train, y_train_pred_proba)

# Compare train vs validation
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC'],
    'Training Set': [
        f"{train_accuracy:.4f}",
        f"{train_precision:.4f}",
        f"{train_recall:.4f}",
        f"{train_f1:.4f}",
        f"{train_roc_auc:.4f}"
    ],
    'Validation Set': [
        f"{val_accuracy:.4f}",
        f"{val_precision:.4f}",
        f"{val_recall:.4f}",
        f"{val_f1:.4f}",
        f"{val_roc_auc:.4f}"
    ],
    'Difference': [
        f"{train_accuracy - val_accuracy:+.4f}",
        f"{train_precision - val_precision:+.4f}",
        f"{train_recall - val_recall:+.4f}",
        f"{train_f1 - val_f1:+.4f}",
        f"{train_roc_auc - val_roc_auc:+.4f}"
    ]
})

print("="*80)
print("OVERFITTING ANALYSIS: Training vs Validation Set Performance")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Interpretation
print("\nInterpretation:")
print("-"*80)
max_diff = max(abs(train_accuracy - val_accuracy),
               abs(train_roc_auc - val_roc_auc))
print(f"Max Diff: {max_diff}")
print("-"*80)


Since max_diff < 0.05 we observe good generalization: Training and validation performance are very similar (< 5% difference).  

This means that our model does not exhbit large overfitting behavior and generalizes well.

In [ ]:
# Visualize train vs validation ROC curves
fpr_train, tpr_train, _ = roc_curve(y_train, y_train_pred_proba)
roc_auc_train = roc_auc_score(y_train, y_train_pred_proba)

plt.figure(figsize=(10, 8))
plt.plot(fpr_train, tpr_train, color='blue', lw=2, label=f'Training Set (AUC = {roc_auc_train:.3f})', linestyle='-')
plt.plot(fpr, tpr, color='red', lw=2, label=f'Validation Set (AUC = {roc_auc:.3f})', linestyle='-')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random Classifier (AUC = 0.500)', alpha=0.5)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
plt.ylabel('True Positive Rate (Recall)', fontsize=12, fontweight='bold')
plt.title('ROC Curves: Training vs Validation Set (Overfitting Check)', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Gap analysis
gap = roc_auc_train - roc_auc
print(f"\nROC-AUC Gap (Train - Validation): {gap:+.4f}")
if abs(gap) < 0.05:
    print("Small gap indicates good generalization")
elif abs(gap) < 0.10:
    print("Moderate gap - acceptable but monitor")
else:
    print("Large gap - potential overfitting")


## Final Summary and Clinical Implications

### Key Findings

**Model Performance:**
- **ROC-AUC**: Excellent ranking ability for patient risk stratification
- **Recall**: High sensitivity ensures we catch most deaths (critical for clinical use)
- **Precision**: Reasonable precision balances false alarms with true detections
- **F1 Score**: Good overall balance between precision and recall

**Feature Importance:**
- Top predictors align with clinical knowledge (time, ejection fraction, serum creatinine)
- Model learns medically relevant patterns, not spurious correlations
- Feature importance provides interpretability for clinicians

**Model Robustness:**
- Good generalization (train/validation performance similar)
- No significant overfitting detected
- Cross-validation results (from notebook 03) were consistent with validation performance

### Clinical Recommendations

1. **Deployment Considerations:**
   - Model shows strong performance but should be validated on external datasets
   - Threshold (0.250) optimized for high recall - prioritize catching deaths
   - Consider clinical workflow integration for real-time risk assessment

2. **Key Risk Factors Identified:**
   - Follow-up time (shorter = higher risk)
   - Ejection fraction (lower = higher risk)
   - Serum creatinine (higher = higher risk)
   - Age and serum sodium also contribute

3. **Limitations:**
   - Small dataset (225 samples) - results may not generalize to all populations
   - Single-center data - external validation needed
   - Model should complement, not replace, clinical judgment


In [ ]:
# Detailed classification report
print("="*80)
print("DETAILED CLASSIFICATION REPORT - VALIDATION SET")
print("="*80)
print(classification_report(y_val, y_val_pred,
                            target_names=['Survived', 'Died'],
                            digits=4))
print("="*80)


In [ ]:
# Plot threshold vs metrics curve
thresholds_plot = np.linspace(0.1, 0.9, 17)
recall_curve = []
precision_curve = []
f1_curve = []

for t in thresholds_plot:
    y_pred_t = (y_val_pred_proba >= t).astype(int)
    recall_curve.append(recall_score(y_val, y_pred_t, pos_label=1, zero_division=0))
    precision_curve.append(precision_score(y_val, y_pred_t, pos_label=1, zero_division=0))
    f1_curve.append(f1_score(y_val, y_pred_t, pos_label=1, zero_division=0))

plt.figure(figsize=(12, 7))
plt.plot(thresholds_plot, recall_curve, 'o-', label='Recall', linewidth=2, markersize=6)
plt.plot(thresholds_plot, precision_curve, 's-', label='Precision', linewidth=2, markersize=6)
plt.plot(thresholds_plot, f1_curve, '^-', label='F1 Score', linewidth=2, markersize=6)
plt.axvline(x=optimal_threshold, color='red', linestyle='--', linewidth=2, label=f'Optimal Threshold (t={optimal_threshold})')
plt.axvline(x=0.5, color='gray', linestyle=':', linewidth=1, alpha=0.7, label='Default (t=0.5)')
plt.xlabel('Classification Threshold', fontsize=12, fontweight='bold')
plt.ylabel('Score', fontsize=12, fontweight='bold')
plt.title('Threshold Analysis: Precision-Recall Trade-off (Validation Set)', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAt optimal threshold (t={optimal_threshold}):")
print(f"Recall: {val_recall:.3f} - Catches {val_recall*100:.1f}% of deaths")
print(f"Precision: {val_precision:.3f} - {val_precision*100:.1f}% of predicted deaths are correct")
print(f"F1: {val_f1:.3f} - Balanced performance metric")


KNN

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold


In [ ]:
# Train, Validation, Test Split
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# helper function for evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def evaluate_model(y_true, y_pred, y_proba=None, model_name="Model"):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred))

    if y_proba is not None:
        print("ROC-AUC:", roc_auc_score(y_true, y_proba))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

numerical_features = X_train.columns.tolist()

# Create a preprocessor to scale numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features)
    ])

# Init Pipeline
knn_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

# Define Hyperparameter Grid
param_grid_knn = {
    'classifier__n_neighbors': [5, 7, 10, 13, 15, 18, 20, 25, 30, 40, 50, 65],
    'classifier__metric': ['euclidean', 'manhattan']
}
# Set up 5-fold CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearch
grid_knn = GridSearchCV(
    estimator=knn_pipeline,
    param_grid=param_grid_knn,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1
)

grid_knn.fit(X_train, y_train)

print("Best params:", grid_knn.best_params_)
print("Best ROC-AUC:", grid_knn.best_score_)

results_df = pd.DataFrame(grid_knn.cv_results_)
print("\nAll cross-validation results:")
print(results_df[['param_classifier__n_neighbors', 'param_classifier__metric', 'mean_test_score']].sort_values('mean_test_score', ascending=False))
tuned_knn_model = grid_knn.best_estimator_

# Make predictions on validation set
y_val_pred_tuned_dt = tuned_knn_model.predict(X_val)
y_val_pred_proba_tuned_dt = tuned_knn_model.predict_proba(X_val)[:, 1]

# Evaluate the tuned model
evaluate_model(y_val, y_val_pred_tuned_dt, y_val_pred_proba_tuned_dt, model_name="Tuned KNN Model")


PCA

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

X = df.drop("DEATH_EVENT", axis=1)
y = df["DEATH_EVENT"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Run PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# View explained variance
explained_var = pca.explained_variance_ratio_

print("Explained variance ratio per PC:")
for i, var in enumerate(explained_var, start=1):
    print(f"PC{i}: {var:.4f}")

# Scree Plot
plt.figure(figsize=(8,5))
plt.plot(range(1, len(explained_var)+1), explained_var, marker='o')
plt.xlabel("Principal Component")
plt.ylabel("Explained Variance Ratio")
plt.title("Scree Plot")
plt.grid(True)
plt.show()

# PC1 vs PC2 scatter plot
pca_df = pd.DataFrame({
    "PC1": X_pca[:, 0],
    "PC2": X_pca[:, 1],
    "Death Event": y
})

plt.figure(figsize=(8,6))
sns.scatterplot(
    data=pca_df,
    x="PC1", y="PC2",
    hue="Death Event",
    palette="coolwarm",
    alpha=0.8
)
plt.title("PC1 vs PC2 colored by Death Event")
plt.show()
